In [2]:
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import numpy as np
import pandas as pd

import os
spark_home='/usr/local/spark-1.5.2-bin-2.7.1'
os.environ['SPARK_HOME']=spark_home
import sys
sys.path.append(os.path.join(spark_home, 'python'))
sys.path.append(os.path.join(spark_home, 'python/lib/py4j-0.8.2.1-src.zip'))
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.feature import Word2Vec
from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark import SparkContext, StorageLevel
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import *
import os
import re
import pickle
from datetime import *
from csv import reader
from io import StringIO
print( "Finished Imports" )

Finished Imports


In [3]:
sc=SparkContext(master="spark://final-gateway:7077", appName="reddiculous")

In [4]:
sqlContext = SQLContext(sc)

In [ ]:
#raw_rdd=sc.textFile("hdfs://final-gateway/w251/reddit")
#parsed_df=sqlContext.jsonRDD(raw_rdd)
# %time parsed_df=sqlContext.read.json("hdfs://final-gateway/w251/reddit-comments")

In [ ]:
#%time parsed_df.count()

In [ ]:
#%time parsed_df.take(1)

In [ ]:
#%time parsed_df.write.format("parquet").save("hdfs://final-gateway/w251/reddit-comments.parquet")

In [6]:
%time parq_df=sqlContext.read.parquet("hdfs://final-gateway/w251/reddit-comments.parquet")

CPU times: user 1.66 ms, sys: 1.25 ms, total: 2.91 ms
Wall time: 5.07 s


In [7]:
%time parq_df.count()

CPU times: user 2.26 ms, sys: 194 µs, total: 2.45 ms
Wall time: 6.39 s


1659361605

In [5]:
%time parq_df.rdd.map(lambda x: 1).reduce(lambda x,y: x+y)

CPU times: user 214 ms, sys: 69 ms, total: 283 ms
Wall time: 13min 34s


1659361605

In [8]:
sqlContext.registerDataFrameAsTable(parq_df,"reddit")

In [17]:
%time ret=sqlContext.sql("SELECT subreddit,count(*) as ct FROM reddit group by subreddit order by ct desc")

CPU times: user 1.02 ms, sys: 0 ns, total: 1.02 ms
Wall time: 49.8 ms


In [27]:
%time ret=sqlContext.sql("SELECT count(*) from (select distinct(subreddit_id) from reddit) ct")

CPU times: user 1.34 ms, sys: 0 ns, total: 1.34 ms
Wall time: 28.3 ms


In [ ]:
%time ret.take(50)

In [6]:
%time parq_df.take(1)

CPU times: user 4.39 ms, sys: 1.16 ms, total: 5.56 ms
Wall time: 6.22 s


[Row(approved_by=None, archived=True, author=u'Jaws666', author_flair_css_class=None, author_flair_text=None, banned_by=None, body=u'There is already a summary.', body_html=None, controversiality=0, created=None, created_utc=u'1303681415', distinguished=None, downs=0, edited=u'false', gilded=0, id=u'c1qsuc3', likes=None, link_id=u't3_gw680', mod_reports=None, name=u't1_c1qsuc3', num_reports=None, parent_id=u't1_c1qs987', removal_reason=None, replies=None, report_reasons=None, retrieved_on=1427090863, saved=None, score=4, score_hidden=False, subreddit=u'WTF', subreddit_id=u't5_2qh61', ups=4, user_reports=None)]

In [14]:
import graphx
import hashlib

In [36]:
vertex_rdd=graphx.VertexRDD(parq_df.rdd.map(lambda x: (hash(graphx.VertexId(x['id'])), x)))

Py4JError: Trying to call a package.

In [29]:
edge_rdd=parq_df.map(lambda x: (hash(x['id']), hash(x['parent_id'], 1.0)))

In [30]:
graphx.Graph(vertex_rdd, edge_rdd)

TypeError: __init__() takes at most 3 arguments (4 given)